<a href="https://colab.research.google.com/github/miroorc/Capstone-Project/blob/main/summarize_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This file summarizes dataset and saves the summarized file in a google drive. This data was used to test BART model summarized version

In [ ]:
pip install --upgrade datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
from datasets import load_dataset
ecthra = load_dataset("lex_glue", "ecthr_a")

In [ ]:
#select the first 3000 original dataset to save time
ecthra['train']=ecthra['train'].select(range(3000))

In [ ]:
train=ecthra['train']
validation=ecthra['validation']
test=ecthra['test']

In [ ]:
#convert label to one-hot form
import numpy as np
num_classes = 10

def one_hot_encode_multi(labels, num_classes):
    encoded_labels = np.zeros((len(labels),num_classes))
    for i, sublist in enumerate(labels):
        encoded_labels[i, sublist] = 1
    return encoded_labels

In [ ]:
train_one_hot=one_hot_encode_multi(train['labels'],10)
train_one_hot = train_one_hot.tolist()
train=train.add_column(name="labels_one_hot", column=train_one_hot)

In [ ]:
test_one_hot=one_hot_encode_multi(test['labels'],10)
test_one_hot = test_one_hot.tolist()
test=test.add_column(name="labels_one_hot", column=test_one_hot)

In [ ]:
validation_one_hot=one_hot_encode_multi(validation['labels'],10)
validation_one_hot = validation_one_hot.tolist()
validation=validation.add_column(name="labels_one_hot", column=validation_one_hot)

In [ ]:
#clean the dataset to remove meaningless characters
import re
def format_text(text):
    text_format = []
    text=[[re.sub(r"^\d+\.\s*","",s) for s in sublst] for sublst in text]
    for each in text:
        text_format.append(''.join(each))
    return text_format

In [ ]:
train_text_format=format_text(train['text'])
train=train.remove_columns('text')
train=train.add_column(name="text", column=train_text_format)
ecthra['train']=train

In [ ]:
test_text_format=format_text(test['text'])
test=test.remove_columns('text')
test=test.add_column(name="text", column=test_text_format)
ecthra['test']=test

In [ ]:
validation_text_format=format_text(validation['text'])
validation=validation.remove_columns('text')
validation=validation.add_column(name="text", column=validation_text_format)
ecthra['validation']=validation

In [ ]:
ecthra=ecthra.remove_columns('labels')
ecthra=ecthra.rename_column('labels_one_hot', 'labels')

In [ ]:
#use bert model to summarise original dataset
from summarizer import Summarizer

def summarize(example):
    model_sum = Summarizer()
    max_length = 1024
    summarized_text = model_sum(example["text"], max_length=1024)
    example["text"] = summarized_text#[0]['summary_text']
    return example
    # summarized_text = summarizer(example["text"], max_length=512)
    # example["text"] = summarized_text[0]['summary_text']
    # return example

# The map method applies the function to each example in the dataset.
ecthra["train"] = ecthra["train"].map(summarize)

In [ ]:
ecthra["train"].save_to_disk("/content/drive/MyDrive/sum_train1")